이름: 

학번:

# Neural networks with PyTorch

Pytorch의 `nn.module`을 활용하여 만드는 유용한 방법을 학습합니다.

<div style="text-align:center"><img src='https://drive.google.com/uc?export=download&id=1J2SeiPpVJs1-ML2BdLrcxkGGmHpRxIVE' width="250" height="200"> 

### Lego block coding! </div>

In [10]:
import torch
import numpy as np

from torch import nn
import torch.nn.functional as F
from collections import OrderedDict

torch.manual_seed(0)

`nn.Linear`: $Z^{[\ell]} = A^{[\ell-1]}W^T+b$
연산.

해당 layer의 

- 입력 차원 `n_input=30`
- 출력 차원 `n_output=60`


In [11]:
# Example of nn.linear
linear_layer1 = nn.Linear(30, 60)
# -> output을 60으로

In [12]:
A = torch.randn(60, 30)
linear_layer1(A).shape

torch.Size([60, 60])

In [13]:
# B = torch.randn(20,30)
# linear_layer2 = nn.Linear(30,3)
# linear_layer2(B).shape
# linear_layer2.weight.data  # tensor의 형태로 돌려줌


How to get the weights and bias of each `nn.Linear`

In [14]:
# Example of weights
linear_layer1.weight.data = torch.ones_like(linear_layer1.weight)

In [15]:
linear_layer1.weight.shape

torch.Size([60, 30])

### NN example

- input units: 20
- hidden layer: 30, 40
- output units: 3
- activation function: ReLU
- output layer: No activation



In [16]:
# Simple NN construction

class FCN(nn.Module):
    def __init__(self):
        super().__init__()   # FCN class의 __init__ 함수 안에서 super class인 nn.Module의 __init__ 함수를 부른다. 
        
        self.lin1 = nn.Linear(20, 30)
        self.lin2 = nn.Linear(30, 40)
        self.lin3 = nn.Linear(40, 3)  
        self.relu = nn.ReLU(True)   # 메모리 절약
        
    def forward(self, x):   # 함수 이름은 forward로 고정
        x = self.lin1(x)   
        x = self.relu(x)
        x = self.lin2(x)   # 위에서 정의된 lin연산과 relu연산을 번갈아가며 하기 위해서 self 변수를 받음
        x = self.relu(x)
        x = self.lin3(x)
        return x
    


In [17]:
Xtrain = torch.randn(60, 20)   # 60x20 Xtrain 행렬 정의

model = FCN()   # model이라는 이름으로 FCN class 생성

#model(Xtrain)   # 위의 셀에서 함수 이름을 forward로 정의하면 model.forward(Xtrain)와 model(Xtrain)이 같은 결과값을 돌려줌(연속적인 선형결합)
                # 20,30 -> 30,40 -> 40,3 => 20x3 return

In [18]:
model
# __init__ 정의된 변수와 lin 객체들을 돌려줌

FCN(
  (lin1): Linear(in_features=20, out_features=30, bias=True)
  (lin2): Linear(in_features=30, out_features=40, bias=True)
  (lin3): Linear(in_features=40, out_features=3, bias=True)
  (relu): ReLU(inplace=True)
)

In [10]:
# Example of parameters() in models
# param_iterator = model.parameters()

# for param in param_iterator:
#     print(param)

# for문 없이는 iterator 객체 내부의 값을 확인할 수 없음

In [2]:
# nn.Sequential() example

class FCN_seq(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc = nn.Sequential(nn.Linear(20, 30),
                      nn.ReLU(True),
                      nn.Linear(30, 40),
                      nn.ReLU(True),
                      nn.Linear(40, 3)   # nn.Sequential함수가 contents들을 순차적으로 결합
        )
        
    def forward(self, x):
        return self.fc(x)                # 위의 예제에서와 같이 함수이름을 forward로 하면 사용이 용이
        
        
        
# self.lin1 = nn.Linear(20, 30)
# self.lin2 = nn.Linear(30, 40)
# self.lin3 = nn.Linear(40, 3)
# self.relu = nn.ReLU(True)

NameError: name 'nn' is not defined

In [20]:
model_seq = FCN_seq()   # 생성
Xtrain.shape 
model_seq

# model_seq(Xtrain) -> class 내부에 forward 함수를 정의하지 않으면 불가능
# model_seq.fc(Xtrain) -> nn.Sequential에 forward 내포

FCN_seq(
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=40, out_features=3, bias=True)
  )
)

In [21]:
model_seq.fc[0]

# nn.Sequential 인덱싱 가능

Linear(in_features=20, out_features=30, bias=True)

In [22]:
class FCN_seq_v2(nn.Module):
    def __init__(self):
        super().__init__()        
        
        temp = self.fcn_block(20, 30)+self.fcn_block(30, 40)+[nn.Linear(40,1)]  # 아래서 정의된 함수 사용, 마지막 layer는 relu 적용x -> 바로 list로 return
        self.fc = nn.Sequential(*temp)   # temp의 data만 추출
        
        
    def fcn_block(self, in_dim, out_dim):   # input과 output을 변수로 전달
        return [nn.Linear(in_dim, out_dim),   # nn.Linear(in, out)적용한 결과와 relu 결과를
                             nn.ReLU(True)]   # list 형태로 return
        
        
    def forward(self, x):
        return self.fc(x)
        
        
        
# self.lin1 = nn.Linear(20, 30)
# self.lin2 = nn.Linear(30, 40)
# self.lin3 = nn.Linear(40, 3)
# self.relu = nn.ReLU(True)

In [23]:
model_seq_v2 = FCN_seq_v2()
model_seq_v2

FCN_seq_v2(
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=40, out_features=1, bias=True)
  )
)

In [24]:
class FCN_final(nn.Module):
    def __init__(self, in_dim, hlayer, out_dim):
        super().__init__()
                                                     # in=20 hidden=[30,40,50,60] out=70
        l_list = self.fcn_block(in_dim, hlayer[0])   # [20,30]
        
        for l1, l2 in zip(hlayer[:-1], hlayer[1:]):   # [30,40,50,60] -> [30,40,50],[40,50,60] 두 개의 list들을 같은 인덱스끼리 짝지어 in, out에 입력
            l_list = l_list + self.fcn_block(l1, l2)   # 선형결합한 각 결과들을 list에 차례로 추가
        
        l_list = l_list + [nn.Linear(hlayer[-1], out_dim)]   #[60,70]
        
        self.fc = nn.Sequential(*l_list)   # l_list의 내용물을 Sequential함수로 저장
        
        
    def fcn_block(self, in_dim, out_dim):
        return [nn.Linear(in_dim, out_dim),
                             nn.ReLU(True)]   # return list
        
        
    def forward(self, x):
        return self.fc(x)
        
        

In [25]:
hlayer = [30, 40]
in_dim = 20
out_dim= 3

myfcn_final = FCN_final(in_dim, hlayer, out_dim)

In [26]:
myfcn_final

FCN_final(
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=40, out_features=3, bias=True)
  )
)

In [27]:
# Ordered dict example
# nn.Sequential() example

class FCN_seq_ordered_dic(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc = nn.Sequential(OrderedDict([('lin1', nn.Linear(20, 30)),
                      ('relu1', nn.ReLU(True)),
                      ('lin2', nn.Linear(30, 40)),
                      ('relu2',nn.ReLU(True)),
                      ('lin3', nn.Linear(40, 3))
                                            ])
        )
        
    def forward(self, x):
        return self.fc(x)
        
        
        
# self.lin1 = nn.Linear(20, 30)
# self.lin2 = nn.Linear(30, 40)
# self.lin3 = nn.Linear(40, 3)
# self.relu = nn.ReLU(True)

In [28]:
# ModuleList(), ModuleDict()

In [109]:
# state_dict() example
#model.state_dict()
# -> OrderDict 타입으로 이름과 내용을 돌려줌

# Problem Setup

<div style="text-align:center"> <img src='https://drive.google.com/uc?export=download&id=1FRhniwGeeutBSJQRdW6GzshMfDrPz7oJ' width="250" height="200"> </div>
    
Build a Fully connected neural network with

- 3 layers
- 마지막 layer의 unit 수는 `1` 
  - 마지막 layer의 activation은 없음 (linear layer)
- Data feature 수는 `100`

- input unit 수는 data 크기를 보고 맞추세요
- hidden layer의 unit 수는 `[80, 50]`
  - hidden layer의 activation 함수는 ReLU

- model class 명 `myFCN`
  - instance 명 `my_model` 생성
  - `my_model` 출력

## Problem 1

problem setup에서 구성한 neural network을 `nn.Sequential`을 활용하여 생성하세요

In [33]:
## 사용할 data 
batch_size = 30
num_feature = 100

X_train = torch.randn(batch_size, num_feature)

In [34]:
X_train.shape

torch.Size([30, 100])

In [49]:
# Problem 1 코딩 (매 줄마다 주석 필수 )

class myFCN(nn.Module):   # class myFCN 정의 
    def __init__(self, in_dim, h_layer, out_dim):   # in-out unit 과 hidden layer unit을 받습니다.
        super().__init__()                          # myFCN class의 __init__ 함수 안에서 super class인 nn.Module의 __init__ 함수를 부릅니다. 
        
        l_list = self.fcn_block(in_dim, h_layer[0])   # in_dim은 hidden layer unit과 다른 변수로 받기 때문에 첫번째 layer는 fcn_block 함수로 먼저 연산합니다.
        
        for l1, l2 in zip(h_layer[:-1], h_layer[1:]): # h_layer list 중 마지막 layer를 뺀 l1, 첫 번째 layer를 뺀 l2 두 개의 list에서 같은 인덱스끼리 짝지어서
            l_list = l_list + self.fcn_block(l1, l2)  # fcn_block(in, out)에 입력하고 각 결과들을 list에 차례로 추가합니다.
            
        l_list = l_list + [nn.Linear(h_layer[-1], out_dim)]   # 마지막 layer는 ReLU를 적용하지 않기 때문에 바로 list로 저장합니다.
        
        self.fc = nn.Sequential(*l_list)   # l_list의 contents를 Sequential함수로 저장, nn.Sequential함수가 contents들을 순차적으로 결합 합니다.
        
        
    def fcn_block(self, in_dim, out_dim):                    # in_dim 과 out_dim을 변수로 전달합니다.
        return [nn.Linear(in_dim, out_dim), nn.ReLU(True)]   # 변수들의 선형결합 결과와 ReLU를 적용한 결과를 list 형태로 return 합니다.
    
    def forward(self, x):   
        return self.fc(x)   # nn.Sequential을 이용하여 l_list의 data를 self.fc에 저장하고 forward 함수를 정의하여 self.fc를 return하면
                            # instance를 생성했을 떄 사용이 편리합니다.


In [114]:
h_layer = [80, 50]     # hidden layer unit 
in_dim = num_feature   # input은 feature 개수에 의해 결정됩니다.
out_dim= 1             # output은 activation이 없고 unit이 1개 입니다.

my_model = myFCN(in_dim, h_layer, out_dim)   # 이름이 my_model인 instance를 생성합니다.
my_model   

myFCN(
  (fc): Sequential(
    (0): Linear(in_features=100, out_features=80, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=80, out_features=50, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=50, out_features=1, bias=True)
  )
)

In [116]:
# my_model(X_train)

## Problem 2

problem setup에서 구성한 neural network을 `OrderedDict`을 활용하여 생성하세요
- 각 layer의 이름을 주고 생성하세요

In [53]:
# 답작성

In [110]:
class myFCN_order(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc = nn.Sequential(OrderedDict([('lin1', nn.Linear(in_dim, h_layer[0])),
                      ('relu1', nn.ReLU(True)),
                      ('lin2', nn.Linear(h_layer[0], h_layer[1])),
                      ('relu2',nn.ReLU(True)),
                      ('lin3', nn.Linear(h_layer[1], out_dim))
                                            ])
        )
                                                                 # python에 있는 OrderedDict 로 이름이나 숫자를 부여해주면
                                                                    # 보다 쉬운 식별이 가능합니다.
    def forward(self, x):
        return self.fc(x)

In [111]:
my_model = myFCN_order()   # instance 생성
my_model                   # myFCN_order class에서 설정한 이름으로 my_model의 정보를 출력합니다.

myFCN_order(
  (fc): Sequential(
    (lin1): Linear(in_features=100, out_features=80, bias=True)
    (relu1): ReLU(inplace=True)
    (lin2): Linear(in_features=80, out_features=50, bias=True)
    (relu2): ReLU(inplace=True)
    (lin3): Linear(in_features=50, out_features=1, bias=True)
  )
)

In [113]:
my_model.fc.lin1

Linear(in_features=100, out_features=80, bias=True)